In [41]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# taking image path or name from user
img1 = input('Enter Image 1 of Stereo Images: ')
img2 = input('Enter Image 2 of Stereo Images: ')

# Read image
image1 = cv2.imread(img1)
image2 = cv2.imread(img2)

stereo = cv2.StereoSGBM_create(minDisparity=-1, numDisparities=168, blockSize=19)

disparity = stereo.compute(image1, image2)

disparity_map = cv2.normalize(disparity, None, alpha = 0, beta = 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

#Show disparity map before generating 3D cloud to verify that point cloud will be usable. 
cv2.imshow("Disparity", disparity_map)
cv2.waitKey()
cv2.destroyAllWindows()

In [35]:
h,w = image2.shape[:2]

focal_length = 0.6*w

Q = np.float32([[1,0,0,-w/2.0],
    [0,-1,0,h/2.0],
    [0,0,0,-focal_length],
    [0,0,1,0]])


Q2 = np.float32([[1,0,0,0],
    [0,-1,0,0],
    [0,0,focal_length*0.05,0], #Focal length multiplication obtained experimentally. 
    [0,0,0,1]])

def write_ply(fn, verts, colors):
    
    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''
    verts = verts.reshape(-1, 3)
    colors = colors.reshape(-1, 3)
    verts = np.hstack([verts, colors])
    with open(fn, 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')

#Reproject points into 3D
points_3D = cv2.reprojectImageTo3D(disparity_map, Q2)

#Get color points
colors = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

#Get rid of points with value 0 (i.e no depth)
mask_map = disparity_map > disparity_map.min()

#Mask colors and points. 
output_points = points_3D[mask_map]
output_colors = colors[mask_map]

#Define name for output file
output_file = 'reconstructed-test.ply'

#Generate point cloud 
write_ply(output_file, output_points, output_colors)